In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
seed=42

In [2]:
# Load data
from src.data import make_dataset
molecules=make_dataset.load()

RDKit WARNING: [21:05:19] Enabling RDKit 2019.09.2 jupyter extensions


In [3]:
# Build the features
from src.features import build_features
X=build_features.build(molecules)

mean value of features = 0.039648946126302084


In [5]:
# Build the targets
from src.features import build_targets
y=build_targets.build_hit(molecules)


Processed 46 hits.
Matched 46 hits


In [6]:
# Create train/test sets
from src.models import split_data
X_train, X_test, y_train, y_test, ind_train, ind_test = split_data.split(0.4,X,y,seed)
mol_test=[molecules[x] for x in ind_test]
mol_train=[molecules[x] for x in ind_train]
    

Fraction hits: train: 0.049, test: 0.047


In [7]:
# Train model
from src.models import train_model
models=[train_model.train_lr(X_train,y_train,seed),train_model.train_rfc(X_train,y_train,seed,n_estimators=10)]

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
Cross-validation score = 0.784 += 0.135
RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)
Cross-validation score = 0.762 += 0.121


In [8]:
# Test model
from src.models import predict_model
for model in models:
    print("------")
    print(model)
    print("Train:")
    yp_train=predict_model.predict(model,X_train,y_train,mol_train)
    print("Test:")
    yp_test=predict_model.predict(model,X_test,y_test,mol_test)
    print("LOO:")
    yp_loo=predict_model.predictLOO(model,X,y,molecules)



------
LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
Train:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       548
         1.0       1.00      1.00      1.00        28

    accuracy                           1.00       576
   macro avg       1.00      1.00      1.00       576
weighted avg       1.00      1.00      1.00       576

[[548   0]
 [  0  28]]
ROC_AUC Score = 1.000
Test:
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       366
         1.0       0.67      0.56      0.61        18

    accuracy                           0.97       384
   macro avg       0.82      0.77      0.79       384
w